<a href="https://colab.research.google.com/github/stepanjaburek/workingpaper_czech_psp_speeches/blob/main/streamline_translation_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Machine translation using the Opus-MT model from Uni Helsinky**

# Setup

In [ ]:
!pip install transformers sentencepiece sacremoses torch tqdm

import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from tqdm.notebook import tqdm
import torch

In [14]:
def translate_csv(file_path, source_lang='cs', target_lang='en', batch_size=8):
    df=pd.read_csv(file_path)
    model_name = f'Helsinki-NLP/opus-mt-{source_lang}-{target_lang}'
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name).to('cuda' if torch.cuda.is_available() else 'cpu')

    def translate_batch(texts):
        inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
        inputs = {k: v.to(model.device) for k, v in inputs.items()}
        with torch.no_grad():
            return tokenizer.batch_decode(model.generate(**inputs), skip_special_tokens=True)

    translations = []
    for i in tqdm(range(0, len(df), batch_size)):
        translations.extend(translate_batch(df['context_full'][i:i + batch_size].tolist()))

    df['translated_context_full'] = translations
    return df

# Translation left


In [ ]:
translated_df = translate_csv('/content/left.csv')
translated_df.to_csv('left_translated.csv', index=False)

# Translation right

In [ ]:
translated_df = translate_csv('/content/right.csv')
translated_df.to_csv('right_translated.csv', index=False)

# **Machine translation using the NLLB-200 model from Meta**

In [2]:
import pandas as pd
from tqdm import tqdm
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


In [12]:
def translate_csv_nllb(file_path, source_lang='ces_Latn', target_lang='eng_Latn', batch_size=4, max_length=256):

    df = pd.read_csv(file_path)
    model_name = 'facebook/nllb-200-distilled-1.3B'
    tokenizer = AutoTokenizer.from_pretrained(model_name, src_lang=source_lang)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to('cuda' if torch.cuda.is_available() else 'cpu')
    target_token_id = tokenizer.convert_tokens_to_ids(target_lang)

    def translate_batch(texts):
        inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=max_length).to(model.device)
        outputs = model.generate(**inputs, forced_bos_token_id=target_token_id, max_length=max_length)
        return tokenizer.batch_decode(outputs, skip_special_tokens=True)

    df['translated_context_full'] = [
        translation for i in tqdm(range(0, len(df), batch_size))
        for translation in translate_batch(df['context_full'][i:i + batch_size].tolist())
    ]
    return df

In [ ]:
translated_df = translate_csv_nllb('/content/left.csv')
translated_df.to_csv('left_translated.csv', index=False)

In [ ]:
translated_df = translate_csv_nllb('/content/right.csv')
translated_df.to_csv('right_translated.csv', index=False)

# **Sentiment classification using the Political DEBATE model by Burnham et al. (2024)**

# Setup

In [7]:
import torch
from transformers import pipeline
import pandas as pd
from tqdm.notebook import tqdm

In [17]:
def analyze_sentiments(df, classifier, classes, hypothesis, batch_size=16):
    results = []
    for i in tqdm(range(0, len(df), batch_size)):
        batch_output = classifier(
            df['translated_context_full'][i:i + batch_size].tolist(),
            classes,
            hypothesis_template=hypothesis,
            multi_label=False,
            batch_size=batch_size
        )

        for item in batch_output:
            results.append({
                'label': item['labels'][0],
                'score': item['scores'][0],
                **{f'{label}_score': score for label, score in zip(item['labels'], item['scores'])}
            })

    return pd.DataFrame(results)


# Model specification and labeling - Left

In [ ]:
model_name = "mlburnham/Political_DEBATE_large_v1.0"
hypothesis_template =  "The emotional valence of this text towards the political left is {}"
classes = [ "negative", "neutral", "positive"]

device = 0 if torch.cuda.is_available() else -1
classifier = pipeline("zero-shot-classification",
                     model=model_name,
                     device=device)

In [ ]:
df = pd.read_csv("/content/left_translated.csv")
results = analyze_sentiments(df, classifier, classes, hypothesis_template)

pd.concat([df, results], axis=1).to_csv('debate_sentiment_left.csv', index=False)


print("\nSentiment Distribution:")
print(results['label'].value_counts())

# Model specification and labeling - Right

In [ ]:
model_name = "mlburnham/Political_DEBATE_large_v1.0"
hypothesis_template =  "The emotional valence of this text towards the political right is {}"
classes = [ "negative", "neutral", "positive"]

device = 0 if torch.cuda.is_available() else -1
classifier = pipeline("zero-shot-classification",
                     model=model_name,
                     device=device)

In [ ]:
df = pd.read_csv("/content/right_translated.csv")
results = analyze_sentiments(df, classifier, classes, hypothesis_template)


pd.concat([df, results], axis=1).to_csv('debate_sentiment_right.csv', index=False)


print("\nSentiment Distribution:")
print(results['label'].value_counts())

# Model specification and labeling - Topic Modeling Left

In [ ]:
model_name = "mlburnham/Political_DEBATE_large_v1.0"
hypothesis_template = "This text discusses the political left in terms of {}"
classes = ["economic issues",
    "social/cultural issues",
    "institutional issues",
    "historical issues",
    "foreign policy issues"]

device = 0 if torch.cuda.is_available() else -1
classifier = pipeline("zero-shot-classification",
                     model=model_name,
                     device=device)

In [19]:
df = pd.read_csv("/content/left_translated.csv")
results = analyze_sentiments(df, classifier, classes, hypothesis_template)


pd.concat([df, results], axis=1).to_csv('debate_topic_left.csv', index=False)


print("\nSentiment Distribution:")
print(results['label'].value_counts())

  0%|          | 0/88 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



Sentiment Distribution:
label
economic issues           630
institutional issues      269
historical issues         184
social/cultural issues    160
foreign policy issues     156
Name: count, dtype: int64
